<a href="https://colab.research.google.com/github/gbulg/Data-analysis-class/blob/main/HW1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1

## 1. Read the novel text from txt file

Open the txt file, read from it and then close it.

In [28]:
f = open("./goncharov_obryv.txt", "r")
novel = f.read()
f.close()

Print first 500 characters to make sure everything is fine:

In [29]:
print(novel[:500])

Иван Александрович Гончаров

Обрыв

Роман в пяти частях.



ЧАСТЬ ПЕРВАЯ





I





Два господина сидели в небрежно убранной квартире в Петербурге, на одной из больших улиц. Одному было около тридцати пяти, а другому около сорока пяти лет.

Первый был Борис Павлович Райский, второй -- Иван Иванович Аянов.

У Бориса Павловича была живая, чрезвычайно подвижная физиономия. С первого взгляда он казался моложе своих лет: большой белый лоб блистал свежестью, глаза менялись, то загорались мыслию, чувс


---
## 2. Lemmatize text using Mystem and save it

First, import Mystem and create an instance of Mystem class:

In [30]:
from pymystem3 import Mystem

m = Mystem()

Lemmatize the text, and print a little slice of the obtained lemmas list:

In [4]:
lemmas = m.lemmatize(novel)

In [5]:
lemmas[50:60]

['квартира', ' ', 'в', ' ', 'петербург', ', ', 'на', ' ', 'один', ' ']

Now we need to save this lemmatization into the new txt file. I'm not sure how exactly it should be formatted, so I will just join this normal forms of words into new text and save it like that.

Open the file for writing, write into it and close it:

In [6]:
f = open("goncharov_obryv_lemmas.txt", "w")
f.write(''.join(lemmas))
f.close()

The saved text looks like this:

In [7]:
print(''.join(lemmas)[84:284])

два господин сидеть в небрежно убирать квартира в петербург, на один из большой улица. один быть около тридцать пять, а другой около сорок пять год.

первый быть борис павлович райский, второй -- иван


---
## 3 Tokenizing the text with NLTK and use pymorphy to analyze words

First, import NLTK.


In [8]:
import nltk

Tokenize the text and print a little slice of the obtained token list:

In [9]:
tokens = nltk.word_tokenize(novel)

In [11]:
tokens[100:110]

['.', 'Иногда', 'же', 'смотрели', 'они', 'зрело', ',', 'устало', ',', 'скучно']

Now let's analyze these tokens with pymorphy. Import the library and create an instance of it:

In [16]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

Here I define a function `token_to_dict` that analyzes one token and returns a desired dictionary for it, and then create a list of such dictionaries by applying this function to each token. 



In [18]:
def token_to_dict(token):
  analysis = morph.parse(token)[0] # take the analysis with the highest score
  return {"lemma": analysis.normal_form, "word": analysis.word, "pos": analysis.tag.POS}

In [19]:
tokens_analysis = list(map(token_to_dict, tokens))

Now is good time to get rid of all punctuation, as all such tokens have their POS value equal to `None` and can easily be filtered out. I also count the number of tokens before and after the filtering.

In [20]:
len(tokens_analysis)

306236

In [21]:
tokens_analysis = list(filter(lambda tkn: tkn["pos"] != None, tokens_analysis))

In [22]:
len(tokens_analysis)

226709

We can see that around 70,000 tokens were filtered out. And now a short slice of the obtaned array of dicts:

In [23]:
tokens_analysis[100:110]

[{'lemma': 'этот', 'word': 'эти', 'pos': 'ADJF'},
 {'lemma': 'неизгладимый', 'word': 'неизгладимые', 'pos': 'ADJF'},
 {'lemma': 'знак', 'word': 'знаки', 'pos': 'NOUN'},
 {'lemma': 'время', 'word': 'времени', 'pos': 'NOUN'},
 {'lemma': 'и', 'word': 'и', 'pos': 'CONJ'},
 {'lemma': 'опыт', 'word': 'опыта', 'pos': 'NOUN'},
 {'lemma': 'гладкий', 'word': 'гладкие', 'pos': 'ADJF'},
 {'lemma': 'чёрный', 'word': 'чёрные', 'pos': 'ADJF'},
 {'lemma': 'волос', 'word': 'волосы', 'pos': 'NOUN'},
 {'lemma': 'падать', 'word': 'падали', 'pos': 'VERB'}]

Now let's create a `.jsonl` file and save this information there. I import `json` library and use `json.dumps` to turn dicts into JSON strings.

In [32]:
import json

In [35]:
f = open("analyzed_tokens.jsonl", "w", encoding="utf-8")
f.write('\n'.join(json.dumps(tkn, ensure_ascii=False) for tkn in tokens_analysis))
f.close()

---
## 4.1. Calculating the shares of each part of speech

To do so, we first extact a list of all unique values of POS by using `set`. Then, for each part of speech, we calculate it's share by filtering only tokens of this POS, counting them, and dividing their amount by the overall token amount.

In [42]:
parts_of_speech = set(tkn["pos"] for tkn in tokens_analysis)

for pos in parts_of_speech:
  share = len(list(filter(lambda tkn: tkn["pos"] == pos, tokens_analysis))) / len(tokens_analysis) * 100
  print(pos + ":\t" + f"{share:.2f}" + "%")

ADVB:	6.90%
NPRO:	11.71%
VERB:	15.38%
CONJ:	11.15%
INFN:	2.79%
PRCL:	6.58%
PRTS:	0.18%
ADJF:	8.85%
COMP:	0.40%
NOUN:	22.02%
INTJ:	0.13%
PRED:	0.65%
PRTF:	0.66%
NUMR:	0.45%
PREP:	10.09%
GRND:	1.21%
ADJS:	0.86%


We can see that the most frequent categories are Nouns (22%), Verbs (15%), Pronouns (11%), Conjunctives (11%) and Prepositions (10%).

---
## 4.2. Most frequent Verbs and Adverbs

To obtain the top-20 most frequent Verbs and Adverbs, i will use the `Counter` class.

In [52]:
from collections import Counter

First, I construct the `verbs` list, which contains all verb lemmas (with repititions).

Then the `Counter` class allows to obtain two lists: `Counter(verbs).keys()` is a list of unique verb lemmas, `Counter(verbs).values()` is a list of the same size that for each unique lemma contains number of its occurencies in the `verbs` list. 

At last, I sort `Counter(verbs).keys()` in the descending order of corresponding values in `Counter(verbs).values()` by using `zip`. I obtain `sorted_verbs` -- a list of tuples of lemmas and the number of their occurencies, sorted such that the most frequent ones are at the top.

In [68]:
verbs = [tkn["lemma"] for tkn in tokens_analysis if tkn["pos"] == "VERB"]
sorted_verbs = sorted(zip(Counter(verbs).values(), Counter(verbs).keys()), reverse=True)

Print the top-20 verbs:

In [78]:
for count, lemma in sorted_verbs[:20]:
  print(lemma + "\t" + str(count) + " occurencies")

быть	1938 occurencies
сказать	1412 occurencies
говорить	866 occurencies
хотеть	614 occurencies
знать	606 occurencies
мочь	557 occurencies
спросить	427 occurencies
любить	415 occurencies
стать	388 occurencies
видеть	386 occurencies
думать	354 occurencies
пойти	321 occurencies
дать	275 occurencies
смотреть	258 occurencies
сделать	242 occurencies
идти	218 occurencies
заметить	217 occurencies
казаться	205 occurencies
взять	190 occurencies
делать	182 occurencies


All the same for the adverbs.

In [77]:
adverbs = [tkn["lemma"] for tkn in tokens_analysis if tkn["pos"] == "ADVB"]
sorted_adverbs = sorted(zip(Counter(adverbs).values(), Counter(adverbs).keys()), reverse=True)

In [79]:
for count, lemma in sorted_adverbs[:20]:
  print(lemma + "\t" + str(count) + " occurencies")

только	775 occurencies
ещё	627 occurencies
потом	534 occurencies
опять	509 occurencies
теперь	440 occurencies
вдруг	386 occurencies
там	379 occurencies
ничего	375 occurencies
где	317 occurencies
почти	252 occurencies
тут	237 occurencies
зачем	224 occurencies
уже	212 occurencies
уж	196 occurencies
здесь	188 occurencies
никогда	181 occurencies
вон	176 occurencies
тогда	170 occurencies
иногда	166 occurencies
тихо	159 occurencies


# TASK

5. (1 point) Find top-25 bigrams and trigrams for your text (use nltk.bigrams), use only lemmas, get rid of the punctuation. Comment shortly on the results.

6. (2 points) Take 3-8 sentences from the original text and substitute some morphological information, for example, change the tense of the verbs, the number of the nouns, e.g, the original Слон подарил мартышке цветы should become Слоны подарят мартышкам цветок.